## Piping Chains and the RunnablePassthrough Class

In [1]:
import os
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [2]:
os.environ.setdefault("no_proxy", "127.0.0.1,localhost")
os.environ.setdefault("HTTPX_NO_PROXY", "127.0.0.1,localhost")
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://127.0.0.1:11434")

model_name = "gemma3" #"llama3.2:3b"

max_tokens = 100
llm = ChatOllama(model=model_name, num_predict=max_tokens, temperature=0)

In [4]:
RunnablePassthrough().invoke([1, 2, 3])

[1, 2, 3]

In [6]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [7]:
chat_template_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job title} needs?\nAnswer only by listing the tools.\n'), additional_kwargs={})])

In [ ]:
string_parser = StrOutputParser()

In [13]:
chain_tools = (chat_template_tools | llm | string_parser| {'tools':RunnablePassthrough()} )
chain_strategy = chat_template_strategy | llm | string_parser

In [14]:
print(chain_tools.invoke({'job title':'data scientist'}))

{'tools': '1. Python\n2. SQL\n3. Pandas\n4. Jupyter Notebook\n5. Tableau/Power BI'}


In [15]:
print(chain_strategy.invoke({'tools':'''
1. Python
2. SQL
3. Pandas
4. Jupyter Notebook
5. Tableau/Power BI
'''}))

Okay, let's develop a strategic learning plan for these five tools – Python, SQL, Pandas, Jupyter Notebook, and Tableau/Power BI. This plan focuses on a phased approach, prioritizing foundational knowledge and building progressively more complex skills.

**Overall Philosophy:**  Hands-on learning is *crucial*.  Don't just read tutorials; *do* the exercises.  Start small, build incrementally, and focus on understanding the "why" behind the tools, not just the


In [16]:
chain_combined = chain_tools | chain_strategy

In [17]:
print(chain_combined.invoke({'job title':'data scientist'}))

Okay, let's develop a strategic learning plan for these five tools – Python, SQL, Pandas, Jupyter Notebook, and Tableau/Power BI. This plan focuses on a phased approach, prioritizing foundational knowledge and gradually building complexity.

**Overall Philosophy:**  Hands-on learning is *crucial*. Don't just read tutorials; *do* the exercises.  Start small, build incrementally, and focus on understanding the "why" behind the tools.

**Phase 1:


## Graphing Runnables

In [18]:
chain_long = (chat_template_tools | llm | string_parser | {'tools':RunnablePassthrough()} | 
              chat_template_strategy | llm | string_parser)

In [ ]:
# pip install grandalf

chain_long.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOllama |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       